In [1]:
import concurrent.futures
import pandas as pd
import rasterio as rio
import numpy as np
import pickle
import glob
from rasterio.windows import Window
from pyproj import Transformer
from tqdm import tqdm
from pathlib import Path

In [2]:
station_list = pd.read_json("./first_order_station_data/first-order-stations.json")
station_list.set_index("station", inplace=True)
station_data_fps = [x for x in Path("./first_order_station_data/").rglob("*.csv")]
years = [str(x) for x in range(2006, 2023)]
years

['2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022']

In [3]:
cmip5_path = Path("/atlas_scratch/cparr4/backtest/gcms")
gcms = ["5ModelAvg", "GFDL-CM3", "GISS-E2-R", "IPSL-CM5A-LR", "MRI-CGCM3", "CCSM4"]
rcps = ["rcp45", "rcp60", "rcp85"]
clim_vars = ["pr_total_mm", "tas_mean_C"]

cmip5_fps = [x for x in Path(cmip5_path).rglob("*.tif") if x.name.split("_")[-1][:-4] in years]

with rio.open(cmip5_fps[0]) as src:
    tif_crs = src.crs
    transformer = Transformer.from_crs("epsg:4326", tif_crs)

assert len(cmip5_fps) == len(years)*len(gcms)*len(rcps)*len(clim_vars)*12

In [4]:
def parse_model(geotiff_path):
    return [x for x in gcms if x.lower() in geotiff_path.name.lower()][0]


def parse_scenario(geotiff_path):
    return [x for x in rcps if x.lower() in geotiff_path.name.lower()][0]


def parse_clim_var(geotiff_path):
    return [x for x in clim_vars if x.lower() in geotiff_path.name.lower()][0]


def transform_lat_lon(lat, lon):
    yx_3338 = transformer.transform(lat, lon)
    return yx_3338

In [6]:
def extract_point_value(geotiff_path, point):
    with rio.open(geotiff_path) as src:
        
        y, x = point
        row, col = src.index(y, x)
        
        window = Window(col, row, 1, 1)
        arr = src.read(1, window=window)

        year = geotiff_path.name.split("_")[-1][:-4]
        mo = geotiff_path.name.split("_")[-2]
        mo_year = f"{mo}-{year}"
        
        model = parse_model(geotiff_path)
        scenario = parse_scenario(geotiff_path)
        climate_variable = parse_clim_var(geotiff_path)

        return mo_year, model, scenario, climate_variable, arr[0]


def extract_point_values(geotiff_paths, points):
    with concurrent.futures.ProcessPoolExecutor(max_workers=32) as executor:
        # Use tqdm to create a progress bar
        with tqdm(total=len(geotiff_paths)) as pbar:
            # Map the extract_point_value function over the list of geotiff_paths and points
            results = []
            for result in executor.map(extract_point_value, geotiff_paths, points):
                results.append(result)
                pbar.update()
    del executor

    # Create a Pandas DataFrame from the results
    df = pd.DataFrame(results, columns=['date', 'model', 'scenario', 'variable', 'value'])
    return df

In [7]:
def group_data(flat_df):
    groups = flat_df.groupby(['model', 'scenario', "variable", 'date'])["value"].apply(float)
    return groups.to_frame().sort_values(['model', 'scenario', 'variable', 'date'])

In [8]:
di = {}
for fp in station_data_fps:
    station_id = fp.name.split("_")[1]
    station_name = station_list.loc[station_id]["name"]
    di[station_name] = {}
    
    di[station_name]["lat"] = round(station_list.loc[station_id]["latitude"], 4)
    di[station_name]["lon"] = round(station_list.loc[station_id]["longitude"], 4)
    di[station_name]["point_3338"] = transform_lat_lon(di[station_name]["lat"], di[station_name]["lon"])
    
    di[station_name]["ID"] = station_id
    station_df = pd.read_csv(fp, header=4)
    # observed units are inches precip, degrees F
    di[station_name]["total_precip_observed_mm"] = station_df["Monthly Total Precipitation (in)"].values * 25.4
    di[station_name]["tasmean_observed_C"] = (station_df["Monthly Average Mean Temperature (degF)"].values - 32) * (5/9)
    
    # check for nan values
    pr_nan_count = np.isnan(di[station_name]["total_precip_observed_mm"]).sum()
    tasmean_nan_count = np.isnan(di[station_name]["tasmean_observed_C"]).sum()
    
    # Print the count of NaN values
    if pr_nan_count != 0:
        print(f"{station_name} number of pr NaN values: {pr_nan_count}")
        print("Replacing NaNs with mean values...")

    if tasmean_nan_count != 0:
        print(f"{station_name} number of tasmean NaN values: {tasmean_nan_count}")
        print("Replacing NaNs with mean values...")

    # replace nan values with the mean of the observations
    pr_mean = np.nanmean(di[station_name]["total_precip_observed_mm"])
    tasmean_mean = np.nanmean(di[station_name]["tasmean_observed_C"])
 
    # Create a Boolean mask indicating which elements are NaN
    pr_mask = np.isnan(di[station_name]["total_precip_observed_mm"])
    tasmean_mask = np.isnan(di[station_name]["tasmean_observed_C"])
    
    # Replace the NaN values with the mean
    pr_clean = np.where(pr_mask, pr_mean, di[station_name]["total_precip_observed_mm"])
    tasmean_clean = np.where(tasmean_mask, tasmean_mean, di[station_name]["tasmean_observed_C"])
    
    di[station_name]["pr_total_mm"] = pr_clean
    di[station_name]["tas_mean_C"] = tasmean_clean
    
    # Verify the values were replaced
    pr_nan_count = np.isnan(di[station_name]["pr_total_mm"]).sum()
    tasmean_nan_count = np.isnan(di[station_name]["tas_mean_C"]).sum()
    assert(pr_nan_count + tasmean_nan_count == 0)
    # pop the observational data that wasn't clean
    di[station_name].pop("total_precip_observed_mm")
    di[station_name].pop("tasmean_observed_C")
print("All NaN values have been replaced with mean values.")


Delta Junction number of pr NaN values: 7
Replacing NaNs with mean values...
Utqiaġvik number of pr NaN values: 2
Replacing NaNs with mean values...
All NaN values have been replaced with mean values.


In [9]:
for k in di.keys():
    print(f"Processing {k}...")
    df = group_data(extract_point_values(cmip5_fps, [di[k]["point_3338"]] * len(cmip5_fps)))
    di[k]["extracted_data"] = df
    del df
    print(f"Processing {k} Complete.")

Processing Juneau...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7344/7344 [00:04<00:00, 1796.60it/s]


Processing Juneau Complete.
Processing Ketchikan...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7344/7344 [00:04<00:00, 1720.06it/s]


Processing Ketchikan Complete.
Processing Kodiak...


...

Processing Anchorage Complete.
Processing Nome...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7344/7344 [00:06<00:00, 1059.69it/s]


Processing Nome Complete.
Processing Utqiaġvik...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7344/7344 [00:07<00:00, 1028.52it/s]


Processing Utqiaġvik Complete.


# Nine Point Extractions from 20 GB of GeoTIFFs...pretty fast!

In [10]:
with open('extracted_data/ak_station_extractions.pickle', 'wb') as handle:
    pickle.dump(di, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
di["Fairbanks"]

{'lat': 64.8039,
 'lon': -147.8761,
 'point_3338': (290502.5512221125, 1662590.802400273),
 'ID': 'USW00026411',
 'pr_total_mm': array([  5.588,  19.304,   5.588,  10.16 ,   4.572,  18.034,  56.896,
         54.864,  14.224,  14.224,   2.286,  12.192,  12.7  ,   3.048,
          5.08 ,   2.54 ,  21.844,  47.752,  93.218,  38.608,  40.132,
         12.954,   2.794,   7.874,  23.622,   4.572,   2.54 ,  32.258,
         12.7  ,  52.832, 104.648,  67.564,  16.256,  19.304,   7.112,
         12.7  ,  13.208,  14.986,  27.686,   2.286,   1.27 ,  39.37 ,
          1.524,  69.088,  13.462,  12.7  ,   7.874,   9.144,   0.762,
          3.81 ,   2.286,   7.366,   6.096,  34.544,  78.994,  37.084,
         30.226,  11.43 ,  43.434,   5.842,   4.826,  41.148,   3.81 ,
          1.778,   1.016,  44.958,  43.18 ,  40.64 ,  16.51 ,   8.128,
         11.684,  24.13 ,  17.526,  10.668,  18.796,   1.778,  18.542,
         35.306,  43.688,  36.83 ,  13.97 ,  35.052,   7.366,  30.226,
         20.066,  12